Se importa la libreria de Pandas

In [2]:
import pandas as pd

1 Limpiando data
Archivo csv del historico de mundiales
Archivo csv de los encuentros del 2022
Achivo csv de la data faltante de los historicos de mundiales

In [71]:
df_data_historica= pd.read_csv('fifa_worldcup_matches.csv')
df_data_historica

#Encuentros del mundial de Qatar 2022
df_fixture= pd.read_csv('fifa_worldcup_fixture.csv')
df_fixture

#Partidos faltantes del mundial del 70, 2010 y otros
df_data_faltante = pd.read_csv('fifa_worldcup_missing_data.csv')
df_data_faltante

,home,score,away,year
0,Mexico,0–0,Soviet Union,1970
1,Belgium,3–0,El Salvador,1970
2,Soviet Union,4–1,Belgium,1970
3,Mexico,4–0,El Salvador,1970
4,Soviet Union,2–0,El Salvador,1970
...,...,...,...,...
503,Spain,0–1,Switzerland,2010
504,Chile,1–0,Switzerland,2010
505,Spain,2–0,Honduras,2010
506,Chile,1–2,Spain,2010


1.1 Limpieza de los datos del "df_fixture"

In [72]:
df_fixture #Reviso la data
df_fixture['home'] #Esta alineado hacia la derecha y existen espacios despues de la r de Qatar y así sucesivamente 
df_fixture['home'] = df_fixture['home'].str.strip() #Accedo a la cadena de texto con str y elimino los espacios con strip()
df_fixture['away'] = df_fixture['away'].str.strip()

1.2 Limpieza de df_missing_data y se agrega a df_data_historica

In [73]:
#df_data_faltante[df_data_faltante['home'].isnull()] #con el metodo isnull se valida si hay data faltante
df_data_faltante.dropna(inplace=True) #con el metodo dropna elimino los NaN y con inplace grabo la información

In [74]:
#df_data_faltante[df_data_faltante['home'].isnull()] #con el metodo isnull se valida si hay data faltante

,home,score,away,year


In [75]:
#El objetivo ahora es combinar la data faltante en el archivo de la data historica
#El metodo .concat ayuda a combinar la data faltante con la data historica

df_data_historica = pd.concat([df_data_historica, df_data_faltante], ignore_index=True) #la funcion ignore_index ayuda a ignorar el index que tiene cada uno en su data original
#df_data_historica

#Se eliminan datos duplicados de la data faltante que puedan estar repetidos en la data historica
df_data_historica.drop_duplicates(inplace=True)
df_data_historica.sort_values('year', inplace= True) #con .sort_values() se ordenan los valores y se elige la variable por la cual se van a guiar
df_data_historica

,home,score,away,year
0,France,4–1,Mexico,1930
17,Uruguay,4–2,Argentina,1930
16,Uruguay,6–1,Yugoslavia,1930
15,Argentina,6–1,United States,1930
14,Paraguay,1–0,Belgium,1930
...,...,...,...,...
419,Brazil,2–0,Costa Rica,2018
420,Serbia,1–2,Switzerland,2018
421,Serbia,0–2,Brazil,2018
408,France,1–0,Peru,2018


1.3 Limpieza de la data historica

In [76]:
#Se revisa la data de partidos, que hayan tenido intervención arbitral ej: ausencia de un equipo se asigna resultado de 3-0. Este partido por conocimiento publico es el de Sweden vs Ausstria
#df_data_historica[df_data_historica['home'].str.contains('Sweden') & df_data_historica['away'].str.contains('Austria')]
df_data_historica[df_data_historica['score'].str.contains('w/o')]

#con la función .index me da el index
index_eliminar = df_data_historica[df_data_historica['score'].str.contains('w/o')].index

In [77]:
#Se elimina el index
df_data_historica.drop(index=index_eliminar, inplace=True)

In [78]:
#Valido la eliminación del partido mencionado anteriormente
df_data_historica[df_data_historica['score'].str.contains('w/o')]

,home,score,away,year


In [80]:
#Realizo una expresión regular para identificar los valores que no tengan un digito e identificar los valores
#df_data_historica[df_data_historica['score'].str.contains('[^\d–]')] #Nota: Tuve que copiar el guión de la table para obtener la información de los partidos que se fueron a alargue

#Implemento una expresión regular para eliminar la información diferente de Valor-Valor en la columna 'score'
df_data_historica['score'] = df_data_historica['score'].str.replace('[^\d–]','', regex=True ) #Coloca un espacio '' 

In [81]:
#Valido la ejecución del paso anterior
df_data_historica[df_data_historica['score'].str.contains('[^\d–]')] #Se elimina la información de tiempo de alargue extra, de modo que quede solamente Valor-Valor en score

,home,score,away,year


In [82]:
#df_data_historica['home'] = df_data_historica['home'].str.strip() #Accedo a la cadena de texto con str y elimino los espacios con strip()
#df_data_historica['away'] = df_data_historica['away'].str.strip()

In [83]:
#Procedo a separar los goles del local y el visitante de score, cada uno en una columna
df_data_historica[['HomeGoals', 'AwayGoals']] = df_data_historica['score'].str.split('–', expand=True) #El metodo .spli() permite realizar la separación, al identificar el valor del guion|El parametro expand permite que se creen las dos columnas

In [85]:
#Se crean las dos nuevas columnas
df_data_historica
#df_data_historica.drop('score', axis=1, inplace=True) #Elimino la columna de score #axis indica la columna

In [86]:
#Renombrar los nombres de las columnas
df_data_historica.rename(columns={'home':'HomeTeam',
                                  'away':'AwayTeam', 'year':'Year'}, inplace=True)
df_data_historica


,HomeTeam,AwayTeam,Year,HomeGoals,AwayGoals
0,France,Mexico,1930,4,1
17,Uruguay,Argentina,1930,4,2
16,Uruguay,Yugoslavia,1930,6,1
15,Argentina,United States,1930,6,1
14,Paraguay,Belgium,1930,1,0
...,...,...,...,...,...
419,Brazil,Costa Rica,2018,2,0
420,Serbia,Switzerland,2018,1,2
421,Serbia,Brazil,2018,0,2
408,France,Peru,2018,1,0


In [89]:
#Ahora procedemos a cambiar el tipo de data
#df_data_historica.dtypes #con .dtypes valido los tipos de data
#df_data_historica = df_data_historica.astype({'HomeGoals':int, 'AwayGoals':int, 'Year':int})
df_data_historica.dtypes

HomeTeam     object
AwayTeam     object
Year          int32
HomeGoals     int32
AwayGoals     int32
dtype: object

In [91]:
#Para validar el numero total de goles en un partido
df_data_historica['TotalGoals'] = df_data_historica['HomeGoals'] + df_data_historica['AwayGoals']

In [92]:
df_data_historica

,HomeTeam,AwayTeam,Year,HomeGoals,AwayGoals,TotalGoals
0,France,Mexico,1930,4,1,5
17,Uruguay,Argentina,1930,4,2,6
16,Uruguay,Yugoslavia,1930,6,1,7
15,Argentina,United States,1930,6,1,7
14,Paraguay,Belgium,1930,1,0,1
...,...,...,...,...,...,...
419,Brazil,Costa Rica,2018,2,0,2
420,Serbia,Switzerland,2018,1,2,3
421,Serbia,Brazil,2018,0,2,2
408,France,Peru,2018,1,0,1


2 Se exportan los dataframe limpios

In [93]:
df_data_historica.to_csv('clean_fifa_worldcup_matches.csv', index=False) #No se exportan los index, solo la data
df_fixture.to_csv('clean_fifa_worldcup_fixture.csv', index=False)